In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None

In [4]:
full_df  = pd.read_csv("C:\\Users\\Shalya\\Downloads\\customer-support-on-twitter\\twcs\\twcs.csv", encoding='latin1')

In [5]:
full_df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [6]:
df = full_df[['text']]
df['text'] = df['text'].astype(str)
df.head()

,text
0,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...
4,@sprintcare I did.


In [7]:
df.dtypes

text    object
dtype: object

In [8]:
#Lowercasing
df['text_lower'] = df['text'].str.lower()

In [9]:
df.head()

,text,text_lower
0,@115712 I understand. I would like to assist y...,@115712 i understand. i would like to assist y...
1,@sprintcare and how do you propose we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,@sprintcare i have sent several private messag...
3,@115712 Please send us a Private Message so th...,@115712 please send us a private message so th...
4,@sprintcare I did.,@sprintcare i did.


In [10]:
##droping the lower case column created
df.drop(['text_lower'], axis = 1, inplace=True)

In [11]:
##Removing punctuation

PUNCT_TO_REMOVE = string.punctuation

In [12]:
PUNCT_TO_REMOVE

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
def remove_punctuation(text):
    return text.translate(str.maketrans('','',PUNCT_TO_REMOVE))

In [14]:
df['text_wo_punct'] = df['text'].apply(lambda text: remove_punctuation(text))

In [15]:
df.head()

,text,text_wo_punct
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...
4,@sprintcare I did.,sprintcare I did


In [16]:
## removing stopwords

from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in str(text).split() if word not in STOPWORDS])

In [17]:
df['text_wo_stop'] = df['text_wo_punct'].apply(lambda text: remove_stopwords(text))

In [18]:
df.head()

,text,text_wo_punct,text_wo_stop
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...
4,@sprintcare I did.,sprintcare I did,sprintcare I


In [29]:
## Removing most frequent words

from collections import Counter

cnt = Counter()

for text in df['text_wo_stop'].values:
    for word in text.split():
        cnt[word] +=1

cnt.most_common(10)


[('I', 833982),
 ('us', 444183),
 ('DM', 326609),
 ('help', 253544),
 ('Please', 232561),
 ('We', 212200),
 ('Hi', 210674),
 ('get', 193941),
 ('please', 167897),
 ('Thanks', 165511)]

In [33]:
##Removing top 10 most common words

FREQWORDS = set([w for (w,wc) in cnt.most_common(10)])

def remove_freqwords(text):
    return ' '.join([text for text in str(text).split() if text not in FREQWORDS])

df['text_wo_stopfreq'] = df['text_wo_stop'].apply(lambda text : remove_freqwords(text))

In [35]:
df.head()

,text,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare I did,sprintcare I,sprintcare


In [36]:
cnt = Counter()

for text in df['text_wo_stopfreq'].values:
    for word in text.split():
        cnt[word] +=1

cnt.most_common(10)

[('know', 144712),
 ('like', 143142),
 ('look', 137678),
 ('AmazonHelp', 136686),
 ('sorry', 128866),
 ('service', 126443),
 ('number', 122226),
 ('Im', 121613),
 ('account', 117353),
 ('email', 114799)]

In [37]:
##drop the punct and stopwords removed column

df.drop(['text_wo_punct','text_wo_stop'], axis = 1, inplace = True)

In [38]:
df.head()

,text,text_wo_stopfreq
0,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare


In [45]:
## Removing rare words

n_rare_words = 10

RARE_WORDS = [w for(w, wc) in cnt.most_common()[:-n_rare_words-1:-1]]

In [49]:
def remove_rarewords(text):
    return ' '.join([word for word in str(text).split() if word not in RARE_WORDS])

df['text_wo_stopfreqrare'] = df['text_wo_stopfreq'].apply(lambda text : remove_rarewords(text))

In [50]:
df.head()

,text,text_wo_stopfreq,text_wo_stopfreqrare
0,@115712 I understand. I would like to assist y...,115712 understand would like assist would need...,115712 understand would like assist would need...
1,@sprintcare and how do you propose we do that,sprintcare propose,sprintcare propose
2,@sprintcare I have sent several private messag...,sprintcare sent several private messages one r...,sprintcare sent several private messages one r...
3,@115712 Please send us a Private Message so th...,115712 send Private Message assist Just click ...,115712 send Private Message assist Just click ...
4,@sprintcare I did.,sprintcare,sprintcare


In [19]:
##Stemming
from nltk.stem.porter import PorterStemmer

In [ ]:
df.drop(['text_wo_stopfreq','text_wo_stopfreqrare'], axis =1, inplace =True)

In [21]:
stemmer = PorterStemmer()
def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

df['text_stemmed'] = df['text'].apply(lambda text : stem_words(text))

In [22]:
df.head()

,text,text_wo_punct,text_wo_stop,text_stemmed
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,@sprintcar and how do you propos we do that
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...,@sprintcar I have sent sever privat messag and...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...,@115712 pleas send us a privat messag so that ...
4,@sprintcare I did.,sprintcare I did,sprintcare I,@sprintcar I did.


In [23]:
##Snowball stemmer is used for non english languages .. languages supported are

from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages


('danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

In [27]:
##Lemmatization

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_words(text):
    return ' '. join([lemmatizer.lemmatize(word) for word in str(text).split()])



In [28]:
df['lemmatized_text'] = df['text'].apply(lambda text: lemmatize_words(text))

In [29]:
df.head()

,text,text_wo_punct,text_wo_stop,text_stemmed,lemmatized_text
0,@115712 I understand. I would like to assist y...,115712 I understand I would like to assist you...,115712 I understand I would like assist We wou...,@115712 I understand. I would like to assist y...,@115712 I understand. I would like to assist y...
1,@sprintcare and how do you propose we do that,sprintcare and how do you propose we do that,sprintcare propose,@sprintcar and how do you propos we do that,@sprintcare and how do you propose we do that
2,@sprintcare I have sent several private messag...,sprintcare I have sent several private message...,sprintcare I sent several private messages one...,@sprintcar I have sent sever privat messag and...,@sprintcare I have sent several private messag...
3,@115712 Please send us a Private Message so th...,115712 Please send us a Private Message so tha...,115712 Please send us Private Message assist J...,@115712 pleas send us a privat messag so that ...,@115712 Please send u a Private Message so tha...
4,@sprintcare I did.,sprintcare I did,sprintcare I,@sprintcar I did.,@sprintcare I did.


In [30]:
lemmatizer.lemmatize('running')

'running'

In [31]:
lemmatizer.lemmatize('running','v')

'run'

In [32]:
lemmatizer.lemmatize('stripes','v')


'strip'

In [33]:
lemmatizer.lemmatize('stripes','n')

'stripe'

In [35]:
##lemmatizing with pos tag

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, 'R': wordnet.ADV}

def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return ' '.join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

In [ ]:
df['text_lemmatized'] = df['text'].apply(lambda text : lemmatize_words(text))

In [ ]:
df.head()

In [ ]:
# Removing emojis
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_emoji("game is on 🔥🔥")

In [ ]:
##removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

remove_emoticons("Hello :-)")

In [ ]:
remove_emoticons("I am sad :(")

In [ ]:
#converting emoticons into words as they play important role in sentiment analysis
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

text = "Hello :-) :-)"
convert_emoticons(text)

In [ ]:
text = "I am sad :()"
convert_emoticons(text)

In [ ]:
##convetint emoji's into text
def convert_emojis(text):
    for emot in UNICODE_EMO:
        text = re.sub(r'('+emot+')', "_".join(UNICODE_EMO[emot].replace(",","").replace(":","").split()), text)
    return text

text = "game is on 🔥"
convert_emojis(text)


In [ ]:
text = "Hilarious 😂"
convert_emojis(text)

In [ ]:
##removing url

def remove_url(text):
    url_pattern = re.compile("https?://\S+")
    return urlpattern.sub(r'', text)

In [ ]:
text = "Driverless AI NLP blog post on https://www.h2o.ai/blog/detecting-sarcasm-is-difficult-but-ai-may-have-an-answer/"
remove_urls(text)

In [ ]:
text = "Please refer to link http://lnkd.in/ecnt5yC for the paper"
remove_urls(text)

In [ ]:
##removing html tags

def html_tags_removal(text):
    html_pattern = re.compile('<.*?')
    return html_pattern.sub(r'',text)

text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>"""

print(remove_html(text))


In [ ]:
##Using beautifulsoup to remove tags

def remove_html(text):
    return BeatifulSoup(text, 'lxml').text

text = """<div>
<h1> H2O</h1>
<p> AutoML</p>
<a href="https://www.h2o.ai/products/h2o-driverless-ai/"> Driverless AI</a>
</div>
"""

print(remove_html(text))

In [ ]:
###Chat words conversation

chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

chat_words_conversion("one minute BRB")